# SOLUTION Tutorial 8: Continuing with the panel data and mixed logit

We will continue with the tomato sauce dataset, showcasing how to create random effects on the coefficients, as opposed to the 'agent' effect (random alternative specific constants).




We have two famous brands of catsup and 3 different package sizes.

A description of the dataset can be found [here](https://www.tandfonline.com/doi/pdf/10.1080/07350015.1994.10524547?casa_token=r4LpjVvgDW4AAAAA:FVG8mEexsQ37tJ2bvk7oxZZ9K_jvvMJ2WxglLzBaHQD0_0REkXmKGsPPxXw_LRGwN3YHY8-L-k8U)

# Description of the dataset

* **id**: household identifiers,
* **choice**: one of heinz41, heinz32, heinz28, hunts32.
* **disp_x**: is there a display for brand X ?
* **feat_x**: is there a newspaper feature advertisement for brand x?
* **price_x**: price of brand x

---
---

# Preparing the environment
*The preparation and dataset loading code is given to the students*

In [ ]:
!pip install biogeme

Load the packages, feel free to change the names.

In [ ]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools
import biogeme.distributions as dist

# Load the dataset

In [ ]:
path = 'https://raw.githubusercontent.com/pmontman/pub-choicemodels/main/data/catsup.csv'
catsup_pd = pd.read_csv(path)


In this case, notice the id variable, that identifies each household, so we have data from many choice situations for each household. We have also different amount of observations per household.

In [ ]:
catsup_pd.head(7)

,id,disp_heinz41,disp_heinz32,disp_heinz28,disp_hunts32,feat_heinz41,feat_heinz32,feat_heinz28,feat_hunts32,price_heinz41,price_heinz32,price_heinz28,price_hunts32,choice
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28
5,1,0,0,0,0,0,0,0,0,5.0,3.0,4.7,3.0,heinz28
6,1,0,0,0,1,0,0,0,1,5.1,3.1,4.6,4.1,heinz28


# Auxiliary function

In [ ]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

# Data cleaning: Preparing the dataset for Biogeme

Encode the choice variable (a string) into numbers with the `factorize` function.
We take a look at the codetable to know how the numbers are mapped to the alternatives. The order of the codetable indicates the association,
0: heinz28, 1: heinz41 and so on.

In [ ]:
catsup_pd['choice'], codetable = catsup_pd['choice'].factorize()

In [ ]:
codetable

Index(['heinz28', 'heinz41', 'heinz32', 'hunts32'], dtype='object')

In [ ]:
catsup_pd['choice']

0       0
1       0
2       0
3       0
4       0
       ..
2793    1
2794    1
2795    3
2796    0
2797    0
Name: choice, Length: 2798, dtype: int64

# Random coeffcients (the Betas)

We will showcase how to make the betas 'random' not only the ASCs.

**We will not consider panel information for the sake of simplicity, you might compare the results as an exercise.**

In [ ]:

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setSilent()


In [ ]:
database = db.Database("catsup", catsup_pd)

We now declare the coefficients in our model, we will make a simple model,
just the alternative-specif constants and the variables disp, feat and price.

In [ ]:
ASC_heinz41 = exp.Beta('ASC_heinz41',0,None,None,0)
ASC_heinz32  = exp.Beta('ASC_heinz32 ',0,None,None,0)
ASC_heinz28 = exp.Beta('ASC_heinz28',0,None,None,0)
ASC_hunts32 = exp.Beta('ASC_hunts32',0,None,None,1)

B_disp = exp.Beta('B_disp',0,None,None,0)
B_feat = exp.Beta('B_feat',0,None,None,0)
B_price = exp.Beta('B_price',0,None,None,0)

We create the parameters that will be the standard deviations, and the Error components.

In [ ]:

SIGMA_price = exp.Beta('SIGMA_price',1,0,None,0)
EC_price = SIGMA_price * exp.bioDraws('EC_price','NORMAL')


**Pay attention to the utility specification**: We add the error component to the coefficient (the Beta) tha multiplies the variable. The interpretation then is that the `B_price`  will be the mean of the random Beta, and `SIGMA_price` will be the standard deviation.

In [ ]:
globals().update(database.variables)
# Definition of the utility functions
V_heinz41 = ASC_heinz41 + B_disp *disp_heinz41 + B_feat * feat_heinz41 + (B_price + EC_price) * price_heinz41
V_heinz32 = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + (B_price  + EC_price) * price_heinz32
V_heinz28 = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + (B_price + EC_price) * price_heinz28
V_hunts32 = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + (B_price + EC_price) * price_hunts32

The rest of the process follows as usual.

In [ ]:
# Associate utility functions with the numbering of alternatives
V = {0: V_heinz28,
     1: V_heinz41,
     2: V_heinz32,
     3: V_hunts32}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

# Conditional to the random variables, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.logit(V,av, choice)


The difference from the usual declaration of the MNL comes now.

We have to do two new steps:

1. Tell biogeme to consider the panel nature of the data.
2. Tell biogeme to calculate the choice probabilities by simulation. This is how we deal with the random parameters. We simulate for the distribution, and the we calculate the likelihood for that distribution.

And Step 2 we take the model and the modifyi it by the expresion `exp.MonteCarlo`. The final log it to take the loglikelihood.

In [ ]:
logprob = exp.log(exp.MonteCarlo(obsprob))

We we are using simulation, we have to tell biogeme how many draws from the distribution are we going to generate. The more draws, the more accurate estimation, but it is compuationally costly.

We also set up a seed, so we can get the same results if the run the notebook again (setting up a seed is a good habit in general)

In [ ]:

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob,numberOfDraws=50, seed=1)




Estimation and results as usual.

In [ ]:

# Estimate the parameters.
results = biogeme.estimate()

We take a look at the results.

In [ ]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz28,2.469856,0.098792,25.000537,0.000000e+00
ASC_heinz32,1.534375,0.071025,21.603329,0.000000e+00
ASC_heinz41,1.388131,0.119689,11.597775,0.000000e+00
B_disp,0.898166,0.106742,8.414400,0.000000e+00
B_feat,0.935092,0.125123,7.473369,7.815970e-14
B_price,-1.470790,0.083752,-17.561221,0.000000e+00
SIGMA_price,0.404057,0.193259,2.090752,3.655027e-02


# Analyzing the random paremeters

So have recovered that the $\beta_{price}$ is a random variable with mean -1.47 and standard deviation $\sigma$ 0.40.
We can 'ask questions' to interpret what this actually means. The type of questions will ask us to calculate something about the distribtuon, such as what is the proportion of individuals in the population that have values for a coefficient in a given range.



In [ ]:
from scipy.stats import norm

mean_B_price = results.getBetaValues()['B_price']
sd_B_price = results.getBetaValues()['SIGMA_price']
mean_B_price, sd_B_price

(-1.4707903499663153, 0.4040565874691901)

#What is the proportion of people in the population that have a positive utility relationship with price?
Even though the mean of the $\beta_{price}$ is negative, because it is random it might happen that *some* people in the population will have a positive value for their $B_{price}$. We can calculate this easily using the normal cumulative distribution function (CDF).
Remember th CDF(x) is is the probability that values for that distribution is < x.
In our case, we want to calculate the prob. that a gaussian with mean -1.47 and sd 0.4 is smaller than 0.

In [ ]:
norm.cdf(   0, loc=mean_B_price, scale=sd_B_price)

0.9998637129088819

So 0.99984 are negative, the proportion that have positive values for $B_{price}$ is

In [ ]:
1- norm.cdf(   0, loc=mean_B_price, scale=sd_B_price)

0.00013628709111812576

About 1 per 10000 in our population prefers to buy more expensive tomato sauces, all other things being equal.

# Comparing mixed specification vs the fixed effects specification

We can also test if making $B_{price}$ random is significantly better. We can compare the likelihoods, the random component has an additional parameter (the sigma for the price).

In [ ]:
V_heinz41 = ASC_heinz41 + B_disp *disp_heinz41 + B_feat * feat_heinz41 + B_price * price_heinz41
V_heinz32 = ASC_heinz32 + B_disp *disp_heinz32 + B_feat * feat_heinz32 + B_price  * price_heinz32
V_heinz28 = ASC_heinz28 + B_disp *disp_heinz28 + B_feat * feat_heinz28 + B_price * price_heinz28
V_hunts32 = ASC_hunts32 + B_disp *disp_hunts32 + B_feat * feat_hunts32 + B_price  * price_hunts32

In [ ]:
# Associate utility functions with the numbering of alternatives
V = {0: V_heinz28,
     1: V_heinz41,
     2: V_heinz32,
     3: V_hunts32}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

# Conditional to the random variables, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.loglogit(V,av, choice)


In [ ]:
biogeme_fixed  = bio.BIOGEME(database,obsprob)
results_fixed = biogeme_fixed.estimate()
results_fixed.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz28,2.425976,0.090695,26.748635,0.000000e+00
ASC_heinz32,1.501251,0.063948,23.476213,0.000000e+00
ASC_heinz41,1.353699,0.114963,11.775050,0.000000e+00
B_disp,0.875593,0.102257,8.562668,0.000000e+00
B_feat,0.908560,0.120177,7.560200,4.019007e-14
B_price,-1.402407,0.056095,-25.000680,0.000000e+00


In [ ]:
def qbus_likeli_ratio_test_bgm(results_complex, results_reference, signif_level):
  return tools.likelihood_ratio_test( (results_complex.data.logLike, results_complex.data.nparam),
                                     (results_reference.data.logLike, results_reference.data.nparam), signif_level)

In [ ]:
results.getGeneralStatistics()

{'Number of estimated parameters': GeneralStatistic(value=7, format=''),
 'Sample size': GeneralStatistic(value=2798, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-3122.2535307349362, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-2516.3855243932776, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=1211.7360126833173, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.19404830529538886, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=0.19180633489449306, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=5046.771048786555, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=5088.327669873133, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.0007334483552227155, format='.4E'),
 'Number of draws': GeneralStatistic(value=50, format=''),
 'Draws generatio

In [ ]:
results_fixed.getGeneralStatistics()

{'Number of estimated parameters': GeneralStatistic(value=6, format=''),
 'Sample size': GeneralStatistic(value=2798, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-3072.019084343948, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-2517.8772495923886, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=1108.2836695031192, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.18038359122690817, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=0.178430478360332, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=5047.754499184777, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=5083.37446011613, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=9.34362282246986e-05, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=2, format='')}

In [ ]:
qbus_likeli_ratio_test_bgm(results, results_fixed, 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=2.9834503982219758, threshold=3.841458820694124)

# Panel data: Advanced stuff (not implemented in Biogeme)

We have seen that we can specify which variable has the 'panel' or 'grouping' information, for example, we can use the `id` variable in this dataset to identify the household, and the estimare a random effects when the individual unit is the household. The general version of this idea is that you could estimate each coefficient considering each group, for example, one can vary per household, other per district, other per person and so on.

# Exercise: What is the proportion of households that react negatively to displaying the catsup product?

In [ ]:

SIGMA_disp = exp.Beta('SIGMA_disp',1,None,None,0)
EC_disp = SIGMA_disp * exp.bioDraws('EC_disp','NORMAL')


In [ ]:
globals().update(database.variables)
# Definition of the utility functions
V_heinz41 = ASC_heinz41 + (B_disp + EC_disp) *disp_heinz41 + B_feat * feat_heinz41 + (B_price ) * price_heinz41
V_heinz32 = ASC_heinz32 + (B_disp + EC_disp) *disp_heinz32 + B_feat * feat_heinz32 + (B_price ) * price_heinz32
V_heinz28 = ASC_heinz28 + (B_disp + EC_disp)*disp_heinz28 + B_feat * feat_heinz28 + (B_price ) * price_heinz28
V_hunts32 = ASC_hunts32 + (B_disp + EC_disp) *disp_hunts32 + B_feat * feat_hunts32 + (B_price ) * price_hunts32

In [ ]:
# Associate utility functions with the numbering of alternatives
V = {0: V_heinz28,
     1: V_heinz41,
     2: V_heinz32,
     3: V_hunts32}

av = {0: 1,
     1: 1,
     2: 1,
     3: 1}

# Conditional to the random variables, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = models.logit(V,av, choice)


In [ ]:
logprob = exp.log(exp.MonteCarlo(obsprob))

In [ ]:

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob,numberOfDraws=50, seed=1)




In [ ]:
biogeme.number_of_draws = 25

In [ ]:

# Estimate the parameters.
results = biogeme.estimate()

In [ ]:
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_heinz28,2.485190,0.097092,25.596147,0.000000e+00
ASC_heinz32,1.539243,0.067641,22.756026,0.000000e+00
ASC_heinz41,1.380139,0.119291,11.569552,0.000000e+00
B_disp,1.038313,0.159611,6.505283,7.754708e-11
B_feat,0.962492,0.127825,7.529791,5.084821e-14
B_price,-1.429782,0.059147,-24.173264,0.000000e+00
SIGMA_disp,1.339898,0.469705,2.852640,4.335773e-03


In [ ]:
norm.cdf( 0, loc=1.038502, scale=1.340501)

0.2192549757348154

</details>